## 8.1 컨볼루션

### 8.1.1 컨볼루션의 역할
- 이산 컨볼루션(discrete convolution) : 2차원 이미지에 가중치 행렬을 스칼라곱을 수행하는 것으로 정의
    * 가중치 행렬 = 커널
  
[특징]  
- 지역성 : 이동된 커널 * 이미지 스칼라곱 
- 평행이동 불변성 : 이미지 전 영역에 대해 동일한 커널 가중치
- 적은 파라미터 : FC와는 달리 컨볼루션에서의 파라미터 수는 이미지 픽셀 수에 의존하지 않음  
    -> 컨볼루션 커널 크기와 모델에서 얼마나 많은 컨볼루션 필터(출력 채널 수)를 쓰는지에 의존
    
## 8.2 컨볼루션 사용해보기
- nn.Conv1d : 시계열용
- nn.Conv2d : 이미지용
- nn.Conv3d : 용적 데이터나 동영상용

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim

In [20]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [3]:
from torchvision import datasets, transforms
data_path = r'C:\Users\knuyh\Desktop\민지\스터디\파이토치 딥러닝 마스터\Minji\Part01\CH07\Data/'

cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [4]:
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label])
          for img, label in cifar10
          if label in [0, 2]]
cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]

In [5]:
connected_model = nn.Sequential(
            nn.Linear(3072, 1024),
            nn.Tanh(),
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Linear(512, 128),
            nn.Tanh(),
            nn.Linear(128, 2))

In [6]:
first_model = nn.Sequential(
                nn.Linear(3072, 512),
                nn.Tanh(),
                nn.Linear(512, 2),
                nn.LogSoftmax(dim=1))

In [7]:
conv = nn.Conv2d(3, 16, kernel_size = 3) # 입력 RGB 3, 출력 피처 16, kernel_size=(3, 3)
conv

Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))

In [8]:
conv.weight.shape, conv.bias.shape # [출력, 입력, 커널, 커널]

(torch.Size([16, 3, 3, 3]), torch.Size([16]))

In [9]:
img, _ = cifar2[0]
output = conv(img.unsqueeze(0))
img.unsqueeze(0).shape, output.shape  # B X C X H X W, 출력의 픽셀 30으로 잘림

(torch.Size([1, 3, 32, 32]), torch.Size([1, 16, 30, 30]))

### 8.2.1 경계 패딩하기
- 패딩 여부 상관없이 weight, bias 크기는 변하지 않음
- 컨볼루션과 이미지 크기 변경 문제를 별도로 분리해 기억해야 하는 것을 하나 줄이는데 도움
- 컨볼루션 구조 자체에 더 신경쓸 수 있다.

In [10]:
conv = nn.Conv2d(3, 1, kernel_size=3, padding=1)
output = conv(img.unsqueeze(0))
img.unsqueeze(0).shape, output.shape

(torch.Size([1, 3, 32, 32]), torch.Size([1, 1, 32, 32]))

In [11]:
conv.weight.shape, conv.bias.shape

(torch.Size([1, 3, 3, 3]), torch.Size([1]))

### 8.2.2 컨볼루션으로 피처 찾아내기
- CH07에서는 weight, bias 랜덤하게 초기화하고, 역전파를 통해 학습되는 파라미터
- 컨볼루션은 직접 가중치 설정 가능

In [12]:
# bias 0으로 제거해 교란 변수 배제
with torch.no_grad() :
    conv.bias.zero_()
    
# 가중치에 상수값 넣어 출력에서의 각 픽셀이 자신의 이웃 픽셀에 대한 평균 가지게
with torch.no_grad() :
    conv.weight.fill_(1.0 / 9.0) # 3 X 3 이웃

In [13]:
conv.bias, conv.weight

(Parameter containing:
 tensor([0.], requires_grad=True),
 Parameter containing:
 tensor([[[[0.1111, 0.1111, 0.1111],
           [0.1111, 0.1111, 0.1111],
           [0.1111, 0.1111, 0.1111]],
 
          [[0.1111, 0.1111, 0.1111],
           [0.1111, 0.1111, 0.1111],
           [0.1111, 0.1111, 0.1111]],
 
          [[0.1111, 0.1111, 0.1111],
           [0.1111, 0.1111, 0.1111],
           [0.1111, 0.1111, 0.1111]]]], requires_grad=True))

In [14]:
output = conv(img.unsqueeze(0))
# 흐려진 이미지 : 각 출력의 픽셀은 자신의 주변 픽셀에 대한 평균이기에 출력 픽셀에서 이러한 상관관계 반영해 픽셀 간의 변화가 부드러워짐

In [15]:
conv = nn.Conv2d(3, 1, kernel_size=3, padding=1)

with torch.no_grad():
    conv.weight[:] = torch.tensor([[-1.0, 0.0, 1.0],
                                   [-1.0, 0.0, 1.0],
                                   [-1.0, 0.0, 1.0]])
    conv.bias.zero_()
    
# 가로로 인접한 두 영역 사이의 수직 경계 탐색

### 8.2.3 깊이와 풀링으로 한 단계 더 인식하기
- 큰 이미지에서 작은 이미지로 다운샘플링  
 : 컨볼루션을 차례로 층층이 쌓으며 동시에 연속적인 컨볼루션 사이의 이미지 다운샘플링
     * 네 개의 픽셀 평균하기 : 평균 풀링
     * 네 개의 픽셀 중 최댓값 : 맥스 풀링 ; 데이터의 3/4 버림
     * stride 하며 컨볼루션 수행하되, n번째 픽셀만 계산

In [16]:
pool = nn.MaxPool2d(2) # 이미지 절반으로 줄임
output = pool(img.unsqueeze(0))
img.unsqueeze(0).shape, output.shape

(torch.Size([1, 3, 32, 32]), torch.Size([1, 3, 16, 16]))

### 8.2.4 우리의 신경망에 적용하기

In [21]:
img.shape

torch.Size([3, 32, 32])

In [17]:
model = nn.Sequential(
        nn.Conv2d(3, 16, kernel_size=3, padding=1),
        nn.Tanh(),
        nn.MaxPool2d(2), # 16 X 16 이미지로 다운샘플링
        nn.Conv2d(16, 8, kernel_size=3, padding=1),
        nn.Tanh(),
        nn.MaxPool2d(2), # 8 X 8
    
        nn.Linear(8 * 8 * 8, 32),  # FC
        nn.Tanh(),
        nn.Linear(32, 2)) # 이진, 멀티 채널의 2차원 피처

## nn.Sequential() : 각 모듈의 출력을 명시적으로 볼 수 없다.

In [18]:
numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list

(18090, [432, 16, 1152, 8, 16384, 32, 64, 2])

In [23]:
model(img.unsqueeze(0))
# 선형 계층의 크기는 8 * 8 * 8 = 512에 의존성을 가진다.
# 모델의 용량을 높이려면 컨볼루션층의 출력 채널 수를 늘려 연결되는 선형 계층도 함께 키워줌
# 런타임 오류

RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x8 and 512x32)

8 X 8 이미지를 512 요소를 가진 1차원 벡터로 차원 정보를 변경하면 오류 해결 !  
마지막 nn.MaxPool2d(2)의 출력에 대해 **view**를 호출하면 해결할 수 있다.  
하지만, nn.Sequential()은 각 모듈의 출력을 명시적으로 볼 수 없기에 불가능  => nn.Flatten

## 8.3 nn.Module 서브클래싱하기
* 계층 뒤에 다른 계층을 붙이는 것보다 더 복잡한 일을 수행하는 모델을 만들려면 유연성을 위해 nn.Sequential 대신 nn.Module 사용
* 먼저 forward 함수 정의하여 모듈로 입력을 전달하고, 출력 반환하게 해야 한다.  
: 모듈의 연산을 정의하는 영역
* 표준 torch 연산을 사용하기만 한다면 자동미분 기능이 자동으로 역방향 경로를 만들어 주어 nn.Module에는 backward가 필요 없다.
* super().__ init __()을 무조건 호출하기 !

### 8.3.1 nn.Module로 정의된 우리의 신경망
* 분류 신경망의 목적은 일반적으로 큰 수의 픽셀을 가진 이미지에서 출발해 정보를 압축해가면서 분류 클래스로 만들어 가는 것
* 중간에 나타나는 값의 개수가 점점 줄어드는 모습  
-> 컨볼루션의 채널 수가 점점 줄어들고, 풀링에서 픽셀 수가 줄어들며 선형 계층에서는 입력 차원보다 낮은 수의 차원을 출력한다. 빠르게 정보를 축소하는 패턴은 제한된 깊이의 신경망과 작은 이미지에 대해서는 잘 동작하지만, 신경망이 깊어질수록 감소는 천천히 일어난다.
* 최초의 컨볼루션에는 입력 크기에 대해 출력 크기가 줄어들지 않는다.  
만일 출력 픽셀 하나를 32개 채널을 가진 벡터로 본다면, 27개 채널(3채널 X 3 X 3 커널 크기를 가지는 컨볼루션)를 가지는 선형 변환에 해당하며 어느 정도 증가하는 셈이다. 따라서 첫 계층은 예외적으로 데이터의 전체 차원을 크게 증가시킨다.

In [24]:
class Net(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        
        self.fc1 = nn.Linear(8*8*8, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 2)
    
    def forward(self, x) :
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8*8*8)  # B X N 벡터로
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out

### 8.3.2 파이토치가 파라미터와 서브모듈을 유지하는 방법
* 서브모듈은 list나 dict 인스턴스에 들어 있으면 안된다. 그렇지 않으면 옵티마이저가 서브모듈과 파라미터를 찾지 못한다. (nn.ModuleList, nn.ModuleDict는 있다.)
* 훈련 과정이 실제 예측과 매우 다른 경우, 별도의 predict 함수를 만드는 게 낫다. 이런 메소드의 호출은 모듈 자체보다는 forward 메소드를 호출하는 것과 비슷해, hook 호출을 놓치게 되고 __call__을 호출하지 않기 때문에, JIT는 사용시 모듈 구조를 볼 수 없다.  
- hook : 매 layer마다 print문을 통해 확인하지 않아도 각 층의 activation/gradient 값 확인 가능

In [25]:
model = Net()

numel_list = [p.numel() for p in model.parameters()]
print(sum(numel_list), numel_list)

18090 [432, 16, 1152, 8, 16384, 32, 64, 2]


### 8.3.3 함수형 API
* 파라미터가 없는 nn.Tanh, nn.MaxPool2d 같은 서브 모듈은 굳이 등록할 필요가 없다. 이들은 view 호출처럼 forward 함수에서 직접 호출하는 것이 더 쉽다.
* '함수형'이란, 내부 상태가 없다는 의미, 즉 출력값이 전적으로 입력 인자에 의존

In [28]:
import torch.nn.functional as F

class Net(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8*8*8, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x) :
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8*8*8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [32]:
model = Net()

model(img.unsqueeze(0)) # torch.Size([1, 3, 32, 32])

tensor([[0.0759, 0.0127]], grad_fn=<AddmmBackward0>)

## 8.4 우리가 만든 컨볼루션 신경망 훈련시키기
* 바깥 루프는 epoch 단위로 돌며, 안쪽 루프는 Dataset에서 배치를 만드는 DataLoader 단위로 돈다.

1) 모델에 입력값을 넣고 (순방향 전달)
2) 손실값을 계산하고 (순방향 전달)
3) 이전 기울기 값을 0으로 리셋하고
4) loss.backward()를 호출하여 모든 파라미터에 대한 손실값의 기울기를 계산한다.(역방향 전달)
5) 이후 옵티마이저를 통해 손실값을 낮추도록 파라미터를 조정한다.

In [33]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader) :
    for epoch in range(1, n_epochs + 1) :
        loss_train = 0.0
        for imgs, labels in train_loader : # DataLoader가 만들어준 배치 안에서 데이터셋 순화
            outputs = model(imgs) # 모델에 배치 넣어줌
            
            loss = loss_fn(outputs, labels) # 최소화하려는 손실값 계산
            
            optimizer.zero_grad() # 마지막에 이전 기울기 값 지움
            loss.backward() # 역전파, 신경망이 학습할 모든 파라미터에 대한 기울기 계산
            optimizer.step() # 모델 업데이트
            
            loss_train += loss.item() # epoch 동안 확인한 손실값 모두 더함. 기울기값 꺼내고자 .item() 사용해 손실값을 파이썬 수로 변환 !
            
    if epoch == 1 or epoch % 10 == 0:
            print('Epoch {}, Training loss {}'.format(
                epoch, loss_train / len(train_loader)))  # 배치 단위의 평균 손실값 구하기 위해 훈련 데이터 로더의 길이로 나눔

In [34]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=True)

model = Net()
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

In [35]:
training_loop(n_epochs = 100, optimizer = optimizer, model = model,
             loss_fn = loss_fn, train_loader = train_loader)

Epoch 100, Training loss 0.16623300351914327


### 8.4.1 정확도 측정

In [36]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64, shuffle=False)

In [46]:
def validate(model, train_loader, val_loader) :
    for name, loader in [("train", train_loader), ("val", val_loader)] :
        correct = 0
        total = 0
        
        with torch.no_grad() : # 파라미터 업데이트 안해 기울기 필요 없음
            for imgs, labels in loader :
                outputs = model(imgs) # model에 배치 넣어줌
                _, predicted = torch.max(outputs, dim=1) # 가장 높은 값을 가진 인덱스 출력
                total += labels.shape[0] # 배치 크기만큼 증가
                correct += int((predicted==labels).sum()) # 확률값이 가장 높았던 클래스와 레이블의 실측값 비교 (.item()과 같은 기능하는 int)
        print("Accuracy {}: {:.2f}".format(name , correct / total))

validate(model, train_loader, val_loader)

Accuracy train: 0.93
Accuracy val: 0.89


### 8.4.2 모델 저장하고 불러오기

In [50]:
data_path = r'C:\Users\knuyh\Desktop\민지\스터디\파이토치 딥러닝 마스터\Minji\Part01\CH08\Data/'
torch.save(model.state_dict(), data_path + 'birds_vs_airplanes.pt')
# 모델의 모든 파라미터 저장, 모델 구조는 포함되어 있지 않음

In [52]:
# 저장한 모델 불러오기
loaded_model = Net()
loaded_model.load_state_dict(torch.load(data_path + 'birds_vs_airplanes.pt'))

<All keys matched successfully>

In [56]:
loaded_model 

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=512, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=2, bias=True)
)

### 8.4.3 GPU에서 훈련시키기
* .to 메소드 사용하여 데이터 로더에서 얻은 텐서와 파라미터를 GPU로 옮길 수 있다.
* Module.to : 모듈 인스턴스 자체를 수정
* Tensor.to : 새 텐서 반환

device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))  

In [ ]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader) :
    for epoch in range(1, n_epochs + 1) :
        loss_train = 0.0
    
    for imgs, labels in train_loader :
        imgs = imgs.to(device=device)
        labels = labels.to(device=device)
        outputs = model(imgs)
        loss = loss_fn(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss_train += loss.item()
        
    if epoch == 1 or epoch % 10 == 0:
        print('{} Epoch {}, Training loss {}'.format(
            datetime.datetime.now(), epoch,
            loss_train / len(train_loader)))

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=True)

model = Net().to(device=device) # 모델을 GPU로 옮긴다.
# 모델이나 입력을 GPU로 옮기지 않으면, GPU와 CPU 입력을 섞어 실행하는 연산 지원하지 않는 파이토치라 '동일 디바이스에 존재하지 않음' 에러 발생
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64, shuffle=False)
all_acc_dict = collections.OrderedDict() # 차례대로 Dict에 append

def validate(model, train_loader, val_loader):
    accdict = {}
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.shape[0]
                correct += int((predicted == labels).sum())

        print("Accuracy {}: {:.2f}".format(name , correct / total))
        accdict[name] = correct / total
    return accdict

all_acc_dict["baseline"] = validate(model, train_loader, val_loader)

In [ ]:
loaded_model = Net().to(device=device)
loaded_model.load_state_dict(torch.load(data_path + 'birds_vs_airplanes.pt', map_location=device))
# GPU에 있던 가중치는 GPU로 복구
# 나중에 어떤 디바이스에서 돌릴지 모르니, 신경망을 CPU로 옮긴 후 저장하든가, 파일에서 읽어들인 후 CPU로 옮기든가 해야함
# 이를 간단하게 하려면, 가중치를 로딩할 때 파이토치가 기억하는 디바이스 정보를 덮어씀 -> torch.load 인자에 map_location

## 8.5 모델 설계
### 8.5.1 메모리 용량 늘리기 : 너비
* 신경망의 너비 차원 : 신경망 계층 내의 뉴런 수 혹은 컨볼루션 채널 수에 해당하는 값

In [105]:
# 첫 번째 컨볼루션의 출력 채널 수를 더 크게 하고, 이어지는 계층도 맞춰 키워준다.
# 더 길어진 벡터를 가지게 되니 이에 따라 완전 연결 계층으로 전환되는 forward 함수에도 반영됨
class NetWidth(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 16, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(16*8*8, 32)
        self.fc2 = nn.Linear(32, 2)
    
    def forward(self, x) :
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 16 * 8 * 8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [116]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64, shuffle=False)

In [117]:
model = NetWidth() #.to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(n_epochs=100, optimizer=optimizer, model=model, loss_fn=loss_fn, train_loader=train_loader)
validate(model, train_loader, val_loader)

Epoch 100, Training loss 0.083354063877824
Accuracy train: 0.96
Accuracy val: 0.90


{'train': 0.9639, 'val': 0.8995}

In [118]:
# 파라미터를 init에 전달하고 너비를 파라미터화해서 forward함수에서 view를 호출할 때도 이를 고려한다.
# 모델의 용량을 증가시킴
class NetWidth(nn.Module) :
    def __init__(self, n_chans1 = 32) :
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1//2, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8 * 8 * n_chans1//2, 32)
        self.fc2 = nn.Linear(32, 2)
    
    def forward(self, x) :
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * self.n_chans1//2)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [119]:
model1 = NetWidth(n_chans1=32) # .to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

import collections
all_acc_dict = collections.OrderedDict() # 차례대로 Dict에 append

training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model1,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

all_acc_dict["width"] = validate(model1, train_loader, val_loader)

Epoch 100, Training loss 0.7035258112439684
Accuracy train: 0.49
Accuracy val: 0.50


In [120]:
print(sum(p.numel() for p in model.parameters()))
print(sum(p.numel() for p in model1.parameters()))

38386
38386


용량이 클수록 모델이 다룰 수 있는 입력은 다양해진다. 하지만, 동시에 과적합할 가능성도 커진다. 파라미터가 많아져 입력에서 불필요한 부분까지 기억해버릴 수 있기 때문이다.

샘플 수를 늘리거나 데이터가 충분하지 않다면 동일한 데이터를 인공적으로 수정해 증강시켜야 한다.

<br/>
<br/>
  
### 8.5.2 모델이 수렴하고 일반화하도록 돕는 방법 : 정규화
* 모델 훈련은 중요한 두 단계를 거친다.
    - 최적화 단계 : 훈련셋에 대해 손실값을 줄이는 경우
    - 일반화 단계 : 모델이 훈련셋뿐 아니라 이전에 겪어보지 않은 검증셋 같은 데이터에 대해서도 동작하게 하는 것
    
      
**1) 파라미터 제어하기 : 가중치 페널티**  
일반화를 안정적으로 수행하기 위해 손실값에 정규화 항을 넣는다. 이 정규화 항을 조작하여 모델의 가중치가 상대적으로 작게 만든다. 즉, 훈련을 통해 증가할 수 있는 크기를 제한하는 것이다. 큰 가중치 값에 페널티를 부과하는 셈이다. 손실값은 다소 매끄러운 등고선 형태를 띠는데 개별 샘플에 맞춰 얻는 이득이 상대적으로 줄어들게 된다.
- L1 정규화 : 모델의 모든 가중치 절댓값의 합 (라쏘)
- L2 정규화 : 모델의 모든 가중치에 대한 제곱합 (릿지, 가중치 감쇠)  
둘다 어떤 값으로 범위를 조정하는데, 이 값은 훈련 전 설정하는 하이퍼파라미터

** 파이토치의 SGD 옵티마이저에는 이미 가중치 감쇠(L2) 파라미터(weight_decay)가 있고 2 * lambda 값을 가지며 업데이트 동작시 가중치 감쇠를 수행한다.   
손실값에 가중치의 L2 정규화를 더한 것과 동일하므로, 자동미분과 손실값에서 항을 누적하는 작업을 할 필요가 없다.

In [110]:
def training_loop_l2reg(n_epochs, optimizer, model, loss_fn, train_loader) :
    for epoch in range(1, n_epochs + 1) :
        loss_train = 0.0
        for imgs, labels in train_loader :
            #imgs = imgs.to(device=device)
            #labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
            
            l2_lambda = 0.001
            l2_norm = sum(p.pow(2.0).sum() for p in model.parameters()) # L1 정규화면 pow(2.0)을 abs()로 변환
            loss = loss + l2_lambda + l2_norm
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            loss_train += loss.item()
        
        if epoch == 1 or epoch % 10 == 0:
            print('Epoch {}, Training loss {}'.format(
                epoch, loss_train / len(train_loader)))

In [121]:
model = Net() #.to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop_l2reg(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)
all_acc_dict["l2 reg"] = validate(model, train_loader, val_loader)

Epoch 1, Training loss 3.855360269926156
Epoch 10, Training loss 0.6941726784797231
Epoch 20, Training loss 0.6941726784797231
Epoch 30, Training loss 0.6941726784797231
Epoch 40, Training loss 0.6941726784797231
Epoch 50, Training loss 0.6941726784797231
Epoch 60, Training loss 0.6941726784797231
Epoch 70, Training loss 0.6941726784797231
Epoch 80, Training loss 0.6941726784797231
Epoch 90, Training loss 0.6941726784797231
Epoch 100, Training loss 0.6941726784797231
Accuracy train: 0.50
Accuracy val: 0.50


**2) 입력 하나에 너무 의존하지 않기 : 드롭아웃**  
* 훈련을 반복할 때마다 신경망의 뉴런 출력을 랜덤하게 0으로 만드는 작업
* 매 훈련 때마다 조금씩 다른 뉴런의 토폴로지가 만들어지기에, 신경망이 각 입력 샘플을 암기하려는 기회를 줄이므로 과적합을 방지한다.
* 모델이 피처를 만들려는 것을 교란해서 데이터 증강과 비슷한 효과를 내지만, 증강과 다르게 신경망 전체에 효과를 낸다.
* 비선형 활성 함수와 선형 혹은 여러 계층의 컨볼루션 모듈 사이에 nn.Dropout 모듈 넣을 수 있다.
* 어떤 입력이 0이 될지에 대한 확률을 지정해준다.

In [122]:
class NetDropout(nn.Module) :
    def __init__(self, n_chans1 = 32) :
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv1_dropout = nn.Dropout2d(p = 0.4) # 어떤 입력이 0이 될지 확률 지정
        
        self.conv2 = nn.Conv2d(n_chans1, n_chans1//2, kernel_size=3, padding=1)
        self.conv2_dropout = nn.Dropout2d(p = 0.4)
        
        self.fc1 = nn.Linear(8 * 8 * n_chans1//2, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x) :
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = self.conv1_dropout(out)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = self.conv2_dropout(out)
        out = out.view(-1, 8 * 8 * self.n_chans1//2)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [123]:
model = NetDropout(n_chans1=32) #.to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)
all_acc_dict["dropout"] = validate(model, train_loader, val_loader)

Epoch 100, Training loss 0.2364254861975172
Accuracy train: 0.90
Accuracy val: 0.89


훈련 중에 활성화되고 훈련이 끝난 모델을 제품으로 사용할 때는 그냥 통과하게 두거나 확률값에 0을 넣어준다. (추론시에는 드롭아웃 하면 안됨) Dropout 모듈의 train 속성을 통해 제어하면 된다. nn.Model 서브클래스에 대해 model.train() 혹은 model.eval() 호출로 전환 가능하다.
  
<br/>  
  
**3) 활성 함수 억제하기 : 배치 정규화**  
* 입력 범위를 신경망의 활성 함수로 바꿔서 미니 배치가 원하는 분포를 가지게 하는 것
* 미니 배치 샘플을 통해 중간 위치에서 얻은 평균과 표준편차를 사용하여 중간 입력값을 이동하고 범위를 바꾼다. 모델이 보는 개별 샘플이나 이로 인한 이후의 활성화 단계에서는 랜덤하게 뽑아 만들어진 미니 배치에서 통계에 의존한 값의 이동과 범위 조정이 반영되어 있는 상태다. 이 자체로 데이터 증강인 셈이다.
* 배치 정규화가 드롭아웃을 할 필요를 없애거나 줄여준다.
* 활성 함수의 입력 범위를 조정하는 것이므로, 선형 변환(컨볼루션) 뒤에 위치한다. (컨볼루션 -> 배치 정규화 -> 활성화(tanh) -> pooling)

** 비선형 활성 함수를 사용하면, 함수의 임계 영역에서 입력이 활성 함수에 너무 많이 작용하여 기울기가 소실되고 훈련이 느려지는 상황을 피할 수 있다.

In [124]:
class NetBatchNorm(nn.Module) :
    def __init__(self, n_chans1 = 32) :
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv1_batchnorm = nn.BatchNorm2d(num_features = n_chans1) # 입력 채널의 개수
        
        self.conv2 = nn.Conv2d(n_chans1, n_chans1//2, kernel_size=3, padding=1)
        self.conv2_batchnorm = nn.BatchNorm2d(num_features = n_chans1//2)
        
        self.fc1 = nn.Linear(8 * 8 * n_chans1//2, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x) :
        out = self.conv1_batchnorm(self.conv1(x))
        out = F.max_pool2d(torch.tanh(out), 2)
        out = self.conv2_batchnorm(self.conv2(out))
        out = F.max_pool2d(torch.tanh(out), 2)
        out = out.view(-1, 8 * 8 * self.n_chans1//2)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [125]:
model = NetBatchNorm(n_chans1=32) #.to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)
all_acc_dict["batch_norm"] = validate(model, train_loader, val_loader)

Epoch 100, Training loss 0.013112648032831064
Accuracy train: 0.99
Accuracy val: 0.87


드롭아웃처럼 배치 정규화도 훈련 때와 추론 때는 각기 다르게 동작해야 한다. 추론 시 출력은 모델이 이미 봤던 다른 입력의 통계에 의존하는 특정 입력을 위한 것이 되어서는 안된다.

미니 배치가 실행되면 현재의 미니 배치에 대한 평균과 표준편차를 구하는 것과 더불어 파이토치가 전체 데이터셋에 대한 평균과 표준편차도 대략적으로 업데이트한다. 

model.eval()을 명시하고, 모델이 배치 정규화 모듈을 가지는 경우 추정값을 고정하고 정규화에 사용하기만 한다.동작 중인 추정을 해제하고 다시 미니 배치 통계를 추정해가길 원한다면, 드롭아웃에서와 마찬가지로 model.train()을 호출한다.
  
  
<br/>  

### 8.5.3 더 복잡한 구조를 배우기 위해 깊이 파헤치기 : 깊이
**스킵 커넥션**  
* 모델의 깊이가 더해질수록 훈련은 수렴하기 어려워진다.
* 곱셈이 체인이 길게 이어지는 경우 기울기값에 기여하는 파라미터가 사라져버려서 파라미터 같은 값들이 적절하게 업데이트되지 않기에 효과 없는 훈련 초래할 수 있다.(Vanishing)
  
=> 단순한 트릭으로 매우 깊은 신경망이어도 잘 훈련되는 **잔차 신경망**인 **레즈넷**을 만들었다
* 스킵 커넥션 : 입력을 계층 블럭의 출력에 연결하는 것

In [126]:
# 계층 추가
class NetDepth(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3,
                               padding=1)
        self.conv3 = nn.Conv2d(n_chans1 // 2, n_chans1 // 2,
                               kernel_size=3, padding=1)
        self.fc1 = nn.Linear(4 * 4 * n_chans1 // 2, 32) # 풀링을 한 번 더해 4 X 4
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2) # ReLU  사용
        out = F.max_pool2d(torch.relu(self.conv2(out)), 2)
        out = F.max_pool2d(torch.relu(self.conv3(out)), 2)
        out = out.view(-1, 4 * 4 * self.n_chans1 // 2)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [127]:
model = NetDepth(n_chans1=32) #.to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)
all_acc_dict["depth"] = validate(model, train_loader, val_loader)

Epoch 100, Training loss 0.08072924349385842
Accuracy train: 0.96
Accuracy val: 0.90


In [128]:
# ResNet처럼 스킵 커넥션 추가 : forward 함수 첫 번째 계층의 출력을 세 번째 계층의 입력에 추가
class NetRes(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3,
                               padding=1)
        self.conv3 = nn.Conv2d(n_chans1 // 2, n_chans1 // 2,
                               kernel_size=3, padding=1)
        self.fc1 = nn.Linear(4 * 4 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = F.max_pool2d(torch.relu(self.conv2(out)), 2)
        out1 = out
        out = F.max_pool2d(torch.relu(self.conv3(out)) + out1, 2)
        out = out.view(-1, 4 * 4 * self.n_chans1 // 2)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [129]:
model = NetRes(n_chans1=32) #.to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)
all_acc_dict["res"] = validate(model, train_loader, val_loader)

Epoch 100, Training loss 0.06950008105130712
Accuracy train: 0.97
Accuracy val: 0.90


표준 Feed Forward 경로에 추가적으로 첫 번째 활성 함수의 출력을 마지막 부분의 입력으로 사용하는 것이다. : 아이덴티티 매핑  
* 역전파에서, 스킵 커넥션 혹은 심층 신경망에서의 연속적인 스킵 커넥션은 깊은 쪽에 있는 파라미터를 손실값에 연결하는 역할을 한다. 이로 인해 체인으로 길게 연결된 여러 다른 연산들로 곱해질 기회가 줄어들고, 파라미터에 대한 손실값의 편미분으로 손실값에 대한 기울기에 더욱 직접적으로 관여한다.
* 스킵 커넥션은 특히 훈련 초반 단계에서 수렴에 도움이 된다.
* 스킵 커넥션은 잔차 신경망의 손실값 분포는 동일한 깊이나 너비를 가진 Feed Forward NN보다 훨씬 부드럽다.
  
<br/>

**파이토치로 매우 깊은 모델 만들기**
* 100개 이상의 계층을 담는다면,   
(Conv2d + ReLU + Conv2d) + skip connection 같은 빌딩 블럭을 정의한 후,  
for 루프를 사용하여 신경망을 동적으로 만든다.

In [130]:
class ResBlock(nn.Module):
    def __init__(self, n_chans):
        super(ResBlock, self).__init__()
        self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3,
                              padding=1, bias=False)  # BatchNorm 계층은 편향값의 효과를 상쇄
        self.batch_norm = nn.BatchNorm2d(num_features=n_chans)
        torch.nn.init.kaiming_normal_(self.conv.weight,
                                      nonlinearity='relu')  # 커스텀 초기화 : 표준편차를 가지는 표준 랜덤 요소로 초기화
        torch.nn.init.constant_(self.batch_norm.weight, 0.5) # 배치 정규화는 기본값으로 평균 0과 분산 0.5를 가지는 분포 출력으로 초기화
        torch.nn.init.zeros_(self.batch_norm.bias) # bias는 0으로 텐서 채움

    def forward(self, x):
        out = self.conv(x)
        out = self.batch_norm(out)
        out = torch.relu(out)
        return out + x

깊은 모델을 만들기로 했으므로 블럭에 배치 정규화를 넣어 훈련 도중에 기울기 값이 없어지는 것을 방지한다.

In [131]:
class NetResDeep(nn.Module):
    def __init__(self, n_chans1=32, n_blocks=10):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.resblocks = nn.Sequential(
            *(n_blocks * [ResBlock(n_chans=n_chans1)])) # ResBlock()을 n_blocks번 반복하여 list로 만든 후, 이를 nn.Sequential에 전달
        self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = self.resblocks(out)
        out = F.max_pool2d(out, 2)
        out = out.view(-1, 8 * 8 * self.n_chans1)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

init에서 ResBlock 인스턴스 리스트를 포함한 nn.Sequential(한 블럭의 출력을 다음 블럭의 입력으로 사용 가능)을 만든다. 또한 블럭 내의 모든 파라미터를 Net이 볼 수 있게 해준다. 이렇게 만든 sequential을 forward에서 호출해 100개의 블럭을 거쳐 출력을 만든다.

** resblocks에서 맨 앞의 * 는 iterable의 각 항목을 함수의 인자로 전달하는 unpacking 연산자로, 리스트의 각 요소가 개벽적인 인자로 전달된다.

In [132]:
model = NetResDeep(n_chans1=32, n_blocks=100) #.to(device=device)
optimizer = optim.SGD(model.parameters(), lr=3e-3)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)
all_acc_dict["res deep"] = validate(model, train_loader, val_loader)

Epoch 100, Training loss 0.024132309595692406
Accuracy train: 0.98
Accuracy val: 0.88


### 8.5.4 모델의 성능 비교

In [133]:
all_acc_dict

OrderedDict([('width', {'train': 0.4888, 'val': 0.5005}),
             ('l2 reg', {'train': 0.5, 'val': 0.5}),
             ('dropout', {'train': 0.8975, 'val': 0.886}),
             ('batch_norm', {'train': 0.9861, 'val': 0.8705}),
             ('depth', {'train': 0.965, 'val': 0.9025}),
             ('res', {'train': 0.9685, 'val': 0.9035}),
             ('res deep', {'train': 0.9842, 'val': 0.8765})])

In [ ]:
trn_acc = [v['train'] for k, v in all_acc_dict.items()]
val_acc = [v['val'] for k, v in all_acc_dict.items()]

import matplotlib.pyplot as plt
import numpy as np

width =0.3
plt.bar(np.arange(len(trn_acc)), trn_acc, width=width, label='train')
plt.bar(np.arange(len(val_acc))+ width, val_acc, width=width, label='val')
plt.xticks(np.arange(len(val_acc))+ width/2, list(all_acc_dict.keys()),
           rotation=60)
plt.ylabel('accuracy')
plt.legend(loc='lower right')
plt.ylim(0.7, 1)
plt.savefig('accuracy_comparison.png', bbox_inches='tight')
plt.show()